### Datenanreicherung – Artikeltexte scrapen

Dieses Notebook ruft die Artikeltexte über die zuvor extrahierten Links ab. Verarbeitet werden dabei nur die gefilterten Links aus 04_artikel_links_suchen.ipynb, die bereits einem definierten Keyword (BVG, HVV, MVG etc.) zugeordnet wurden.

Die wichtigsten Schritte:
- Bereinigung der Linkliste 
- Herunterladen der verlinkten HTML-Seiten über die gespeicherten URLs  
- Lokales Scraping der Artikeltexte aus den gespeicherten HTML-Dateien extrahieren
- Speicherung der extrahierten Artikeltexte als CSV-Datei für die spätere Analyse
- Protokollierung potenzieller Verarbeitungsfehler zur Nachvollziehbarkeit
- Bereinigung und Zerlegung der Texte in Wörter


Dieses Skript setzt voraus, dass die HTML-Linkdaten bereits extrahiert und gefiltert wurden (siehe: 04_artikel_links_suchen.ipynb)

#### 1. Import benötigte Pakete

In [1]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)

# HTML-Verarbeitung & Scraping
from bs4 import BeautifulSoup # HTML analysieren
import requests # HTTP-Anfragen
import time # Pausen zwischen HTTP-Anfragen

# Eigene Funktionen (ausgelagert)
import sys  # Systemfunktionen 
sys.path.append("..") # Pfad zu .py Datei
from scripts.datenaufbereitung import load_stopwords
from scripts.cluster_mapping import medium_to_cluster
from scripts.textbereinigung import clean_and_split_text

In [2]:
# Pfade
# Projektverzeichnis
PROJECT_ROOT = r"D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping"

# Linkliste mit Headlines und Keywords
DATAPATH = os.path.join(PROJECT_ROOT, "output", "artikel_links_headlines.csv")

# Output-Pfade
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "output") 

# Bereinigte Linkliste mit Headlines und Keywords
CSV_PATH = os.path.join(OUTPUT_PATH, "artikel_links_headlines_clean.csv")

# HTML-Dateien
HTML_PATH = os.path.join(OUTPUT_PATH, "raw_artikel")
CSV_PATH_HTML = os.path.join(HTML_PATH, "artikel_download_log.csv")
FAILED_PATH_HTML = os.path.join(HTML_PATH, "artikel_download_failed.csv")

# Lokal gescrapete Texte
CSV_PATH_RAW_ARTIKEL = os.path.join(OUTPUT_PATH, "artikel_scraped.csv")
FAILED_PATH_RAW_ARTIKEL = os.path.join(OUTPUT_PATH, "artikel_scraped_failed.csv")

# Wortfrequenzen pro Artikel 
CSV_PATH_COUNTS = os.path.join(OUTPUT_PATH, "artikel_counts.csv")

#### 2. Datenexploration und Bereinigung der Linkliste

In [3]:
# CSV-Dateien einlesen
df_artikel = pd.read_csv(DATAPATH)

In [4]:
# Anzahl Zeilen und Spalten
df_artikel.shape

(5593, 7)

In [5]:
# Spaltennamen
df_artikel.columns

Index(['filename', 'source', 'date', 'url', 'headline', 'cluster', 'keyword'], dtype='object')

In [6]:
# Prüfung fehlende Werte
df_artikel.isnull().sum()

filename    0
source      0
date        0
url         0
headline    0
cluster     0
keyword     0
dtype: int64

In [7]:
# Erste 10 Artikelzeilen anzeigen
df_artikel.head(10)

,filename,source,date,url,headline,cluster,keyword
0,2021-04-01-sz.html,sz,2021-04-01,https://www.sueddeutsche.de/muenchen/muenchen-...,Wartungsarbeiten der MVGEinschränkungen im Apr...,Große Medien,mvg
1,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://checkpoint.tagesspiegel.de/langmeldung...,Zweifel an britischem Impfstoff:Verwaltung drä...,Regional,vhh
2,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://www.tagesspiegel.de/themen/bvg/,BVG,Regional,bvg
3,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,ExklusivStreit um Schutz vor Hackerangriffen:B...,Regional,bvg
4,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,mehr,Regional,bvg
5,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/pruefung-vo...,Prüfung von Cyberabwehr:BVG eskaliert im Konfl...,Regional,bvg
6,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/opposition-...,Opposition attackiert Berliner Wirtschaftssena...,Regional,bvg
7,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/gesellschaft/queer...,BVG-Chefin Kreienkamp über ihr Coming-out:„Vie...,Regional,bvg
8,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,09:51Streit um Schutz vor HackerangriffenBVG z...,Regional,bvg
9,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/themen/bvg/,BVG,Regional,bvg


In [8]:
# Kurze Headlines filtern und anzeigen
kurze_headlines = df_artikel[df_artikel["headline"].str.len() <= 3]
print(kurze_headlines[["headline", "url"]])

     headline                                                url
2         BVG            https://www.tagesspiegel.de/themen/bvg/
9         BVG            https://www.tagesspiegel.de/themen/bvg/
16        BVG            https://www.tagesspiegel.de/themen/bvg/
17        BVG            https://www.tagesspiegel.de/themen/bvg/
23        BVG            https://www.tagesspiegel.de/themen/bvg/
...       ...                                                ...
1800      BVG            https://www.tagesspiegel.de/themen/bvg/
1804      BVG            https://www.tagesspiegel.de/themen/bvg/
4913      BVG         https://www.berliner-zeitung.de/topics/bvg
4915      BVG         https://www.berliner-zeitung.de/topics/bvg
5504       52  https://www.zeit.de/arbeit/2025-04/verdi-bvg-b...

[527 rows x 2 columns]


In [9]:
# Anzahl kurze Headlines nach Medium
kurze_headlines["source"].value_counts()

source
tagesspiegel    524
berliner          2
zeit              1
Name: count, dtype: int64

In [10]:
# 5 Links mit häufigster Nennung beim Tagesspiegel 
df_artikel[df_artikel["source"] == "tagesspiegel"]["url"].value_counts().head(5)

url
https://www.tagesspiegel.de/themen/bvg/                                                                                                                                   528
https://www.tagesspiegel.de/berlin/falsch-gehende-uhren-in-berlin-als-die-bvg-nicht-richtig-tickte/28249414.html                                                           15
https://www.tagesspiegel.de/berlin/plotzlich-war-ich-die-stimme-der-bvg-synchronsprecherin-philippa-jarke-macht-die-ansagen-in-berlins-bussen-und-bahnen-12977182.html     12
https://www.tagesspiegel.de/berlin/kuerzungen-bei-der-bvg-31-berliner-buslinien-fahren-ab-montag-seltener-wegen-corona-und-personalmangel/28600266.html                    11
https://www.tagesspiegel.de/berlin/polizei-justiz/19-jaehriger-erleidet-kopfverletzungen-radfahrer-in-berlin-bei-unfall-mit-bvg-bus-schwer-verletzt/27277756.html          11
Name: count, dtype: int64

In [11]:
# Kurze Headlines entfernen
df_artikel = df_artikel[df_artikel["headline"].str.len() > 3]

In [12]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 5066


In [13]:
# Duplikate prüfen
df_artikel["url"].duplicated().sum()

np.int64(3195)

In [14]:
# Duplikate anzeigen 
df_artikel[df_artikel.duplicated(subset="url", keep=False)].sort_values("url")

,filename,source,date,url,headline,cluster,keyword
3081,2023-11-23-taz.html,taz,2023-11-23,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...,Große Medien,bvg
3082,2023-11-24-taz.html,taz,2023-11-24,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...,Große Medien,bvg
1092,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,Schwan-Alarm am Lietzensee:Berlin errichtet we...,Regional,bvg
1093,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,mehr,Regional,bvg
1363,2022-06-02-tagesspiegel.html,tagesspiegel,2022-06-02,https://checkpoint.tagesspiegel.de/langmeldung...,Regulär und weniger regulär:BVG und Bahn kontr...,Regional,bvg
...,...,...,...,...,...,...,...
2109,2022-12-13-zeit.html,zeit,2022-12-13,https://www.zeit.de/mobilitaet/2022-12/sunglid...,124 Kommentare,Große Medien,hochbahn
1504,2022-07-04-handelsblatt.html,handelsblatt,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,Berliner Verkehrsbetriebe:Gericht,Wirtschaft,bvg
1498,2022-07-04-zeit.html,zeit,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,09:51Berliner Verkehrsbetriebe:Gericht: BVG da...,Große Medien,bvg
2633,2023-05-03-zeit.html,zeit,2023-05-03,https://www.zeit.de/wirtschaft/2023-05/mobilit...,"Podcast: Mobilitätswende: ""Sie wollten einfach...",Große Medien,bvg


In [15]:
# Duplikate entfernen
df_artikel = df_artikel.drop_duplicates(subset="url")

In [16]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 1871


In [17]:
# Anzahl Links nach Medium
df_artikel["source"].value_counts()

source
abendblatt      586
berliner        560
tagesspiegel    408
taz              66
heise            60
sz               38
ntv              36
zeit             24
spiegel          21
welt             20
stern            14
boerse           11
dlf               8
faz               6
tagesschau        4
netzpolitik       4
t3n               2
mm                2
wiwo              1
Name: count, dtype: int64

In [18]:
# Ergebnis als CSV-Datei exportieren
df_artikel.to_csv(CSV_PATH, index=False)
print(f"[INFO] Links gespeichert unter: {CSV_PATH}")

[INFO] Links gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_headlines_clean.csv


#### 3. HTML-Dateien downloaden

In [19]:
# Funktion: HTML-Seite von jedem Artikel-URL herunterladen und speichern
def download_article_pages(df, html_path, csv_log_path, csv_failed_path, delay=2.5):
    
    print("[INFO] Starte HTML-Download der Artikel...")

    # Fortlaufende Dateinamen
    df = df.reset_index(drop=True)
    # Ordner anlegen (falls nicht vorhanden)
    os.makedirs(html_path, exist_ok=True)

    # Ergebnis-Listen vorbereiten
    log_list = []
    failed_list = []
    erfolgreich = 0
    fehlerhaft = 0

    # Jede Zeile im DataFrame durchlaufen
    for index, row in df.iterrows():
        url = row["url"]
        filename = f"artikel_{index:04d}.html"
        file_path = os.path.join(html_path, filename)

        try:
            # HTTP-Anfrage an die URL
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)

            # Wenn erfolgreich (Status 200)
            if response.status_code == 200:
                # HTML-Seite speichern
                # WICHTIG: Als Bytes speichern, damit keine Encoding-Fehler durch falsche Schätzung entstehen
                with open(file_path, "wb") as f:
                    f.write(response.content)

                # Erfolgreiches Ergebnis in Log eintragen
                log_list.append({
                    "filename": filename,
                    "url": url,
                    "status": "erfolgreich",
                    "encoding": response.encoding,  # vom Server geschätztes/geliefertes Encoding (nur Info)
                    "source": row.get("source", ""),
                    "date": row.get("date", ""),
                    "headline": row.get("headline", ""),
                    "keyword": row.get("keyword", ""),
                    "cluster": row.get("cluster", "")
                })

                erfolgreich += 1

            else:
                # Fehlerhafte Anfrage 
                failed_list.append({
                    "filename": None,
                    "url": url,
                    "status": f"Fehler: HTTP {response.status_code}",
                    "encoding": None,
                    "source": row.get("source", ""),
                    "date": row.get("date", ""),
                    "headline": row.get("headline", ""),
                    "keyword": row.get("keyword", ""),
                    "cluster": row.get("cluster", "")
                })
                fehlerhaft += 1

        except Exception as e:
            # Technischer Fehler 
            failed_list.append({
                "filename": None,
                "url": url,
                "status": f"Fehler: {e}",
                "encoding": None,
                "source": row.get("source", ""),
                "date": row.get("date", ""),
                "headline": row.get("headline", ""),
                "keyword": row.get("keyword", ""),
                "cluster": row.get("cluster", "")
            })
            fehlerhaft += 1

        # Kurze Pause zwischen den Downloads
        time.sleep(delay)

    # Speichern
    pd.DataFrame(log_list).to_csv(csv_log_path, index=False, encoding="utf-8")
    pd.DataFrame(failed_list).to_csv(csv_failed_path, index=False, encoding="utf-8")

    # Ergebnis anzeigen
    print(f"[INFO] HTML-Download abgeschlossen.")
    print(f"[INFO] Erfolgreich gespeichert: {erfolgreich}")
    print(f"[INFO] Fehlerhafte Seiten: {fehlerhaft}")

In [20]:
# Herunterladen von HTML-Dateien 
# CSV mit den Artikel-Links laden
df_links = pd.read_csv(CSV_PATH)  

In [21]:
# HTML-Seiten herunterladen
download_article_pages(
    df=df_links,
    html_path=HTML_PATH,
    csv_log_path=CSV_PATH_HTML,
    csv_failed_path=FAILED_PATH_HTML,
    delay=2.5 
)

[INFO] Starte HTML-Download der Artikel...
[INFO] HTML-Download abgeschlossen.
[INFO] Erfolgreich gespeichert: 1797
[INFO] Fehlerhafte Seiten: 74


In [22]:
# Fehlerhafte Seiten laden
df_failed = pd.read_csv(FAILED_PATH_HTML)

# Überblick
print(df_failed["status"].value_counts())

status
Fehler: HTTP 403    50
Fehler: HTTP 404    24
Name: count, dtype: int64


#### 4. Lokales Scraping der Artikeltexte

In [23]:
# Funktion: Lokales Scraping: Zeile aus der CSV-Datei nehmen, die dazugehörige HTML-Datei lokal suchen, sie öffnen, <p>-Text extrahieren und Text oder Fehlermeldung zurückgeben
def scrape_article_local(entry, html_path):
    # Dateiname holen bzw. Fehlmeldung
    fn = entry.get("filename", "")
    if not fn:
        return {**entry, "error": "missing_filename"}
    
    # Dateipfad zusammensetzen
    fp = os.path.join(html_path, fn)
    if not os.path.exists(fp):
        return {**entry, "error": "file_not_found"}
    try:
        # Datei öffnen und HTML parsen
        with open(fp, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "html.parser")
        
        # Absätze finden und Text extrahieren
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text(" ", strip=True) for p in paragraphs)
        
        # Wenn Text gefunden wurde, Dictionary mit allen Infos und Text
        if text.strip():
            return {
                "filename": fn,
                "url": entry.get("url", ""),
                "source": entry.get("source", ""),
                "date": entry.get("date", ""),
                "headline": entry.get("headline", ""),
                "keyword": entry.get("keyword", None),
                "text": text,
                "cluster": medium_to_cluster.get(entry.get("source", ""), "Unbekannt")
            }
        else:
            return {**entry, "error": "empty_text"}
    except Exception as e:
        return {**entry, "error": f"exception: {e}"}

In [24]:
# Artikeltexte extrahieren: Lokales Scrpaing durchführen 
# Log der erfolgreichen Artikel laden
df_links = pd.read_csv(CSV_PATH_HTML)

# Ergebnis-Listen vorbereiten
successful_articles = []
failed_articles = []

# Artikeltexte iterativ abrufen
for entry in df_links.to_dict(orient="records"):
    result = scrape_article_local(entry, HTML_PATH)
    if "error" in result:
        failed_articles.append(result)
    else:
        successful_articles.append(result)

# Ergebnis anzeigen
print(f"[INFO] Erfolgreich: {len(successful_articles)} | Fehlgeschlagen: {len(failed_articles)}")

# Ergebnisse speichern
pd.DataFrame(successful_articles).to_csv(CSV_PATH_RAW_ARTIKEL, index=False)
pd.DataFrame(failed_articles).to_csv(FAILED_PATH_RAW_ARTIKEL, index=False)

[INFO] Erfolgreich: 1792 | Fehlgeschlagen: 5


In [25]:
# Qualitätsprüfung: Ergebnis einlesen
df_scraped = pd.read_csv(CSV_PATH_RAW_ARTIKEL)
df_scraped.head()

,filename,url,source,date,headline,keyword,text,cluster
0,artikel_0000.html,https://www.sueddeutsche.de/muenchen/muenchen-...,sz,2021-04-01,Wartungsarbeiten der MVGEinschränkungen im Apr...,mvg,Auf beiden U-Bahn-Linien kommt es wegen Wartun...,Große Medien
1,artikel_0001.html,https://checkpoint.tagesspiegel.de/langmeldung...,tagesspiegel,2021-04-01,Zweifel an britischem Impfstoff:Verwaltung drä...,vhh,Sie wollen Berlins schnellsten Überblick? Jede...,Regional
2,artikel_0002.html,https://www.tagesspiegel.de/berlin/streit-um-s...,tagesspiegel,2021-04-02,ExklusivStreit um Schutz vor Hackerangriffen:B...,bvg,© Kitty Kleist-Heinrich Man sei „falsch abgebo...,Regional
3,artikel_0003.html,https://www.tagesspiegel.de/berlin/pruefung-vo...,tagesspiegel,2021-04-02,Prüfung von Cyberabwehr:BVG eskaliert im Konfl...,bvg,© Hauke-Christian Dittrich/dpa Trotz hoher Gef...,Regional
4,artikel_0004.html,https://www.tagesspiegel.de/berlin/opposition-...,tagesspiegel,2021-04-02,Opposition attackiert Berliner Wirtschaftssena...,bvg,© Fabian Sommer/dpa Berlins Wirtschaftssenator...,Regional


In [41]:
# Qualitätsprüfung: Text anzeigen artikel_0000.html
text_mvg = df_scraped[df_scraped["filename"].isin(["artikel_0000.html"])]["text"].iloc[0]

# Erste 1.000 Zeichen ausgeben
print(text_mvg[:1000])

Auf beiden U-Bahn-Linien kommt es wegen Wartungsarbeiten zu Behinderungen. Worauf sich Fahrgäste einstellen müssen. Wegen Wartungsarbeiten kommt es im U-Bahn-Netz zu Einschränkungen. Zwischen Münchner Freiheit und Alte Heide erneuern die Stadtwerke auf der Linie U 6 von Montag, 5. April, bis Donnerstag, 29. April, die Stromschiene. Von 23 Uhr an gilt: Zwischen Garching Forschungszentrum und Münchner Freiheit fährt die U 6 im 20-Minuten-Takt. Zwischen Implerstraße und Klinikum Großhadern fährt sie alle zehn Minuten. Zwischen Münchner Freiheit und Implerstraße müssen die Fahrgäste auf die U 3 ausweichen. An den Bahnhöfen Münchner Freiheit, Dietlindenstraße, Nordfriedhof und Alte Heide fahren die Züge in beiden Richtungen von Gleis zwei. Ausgenommen von den Arbeiten sind die Nächte von Freitag auf Samstag und von Samstag auf Sonntag. Die U 3 wird wegen Schienenschleifarbeiten von Sonntag, 4. April, bis Donnerstag, 8. April, ab 22.30 Uhr zwischen Fürstenried West und Aidenbachstraße durch 

In [44]:
# Qualitätsprüfung: Text anzeigen artikel_0001.html
text_vhh = df_scraped[df_scraped["filename"].isin(["artikel_0001.html"])]["text"].iloc[0]

# Erste 1.000 Zeichen ausgeben
print(text_vhh[:1000])

Sie wollen Berlins schnellsten Überblick? Jeden Morgen die wichtigsten Nachrichten der Stadt – mit uns verpassen Sie nichts! ...aber jetzt, wo Sie schon da sind: Testen Sie die Checkpoint Kurzstrecke und lesen Sie Berlins beliebtesten Newsletter mit allen wichtigen Nachrichten und Aufregern der Stadt. Gratis. Von Herausgeber Lorenz Maroldt Sie wollen Berlins schnellsten Überblick? Jeden Morgen die wichtigsten Nachrichten der Stadt – mit uns verpassen Sie nichts! ...aber jetzt, wo Sie schon da sind: Testen Sie die Checkpoint Kurzstrecke und lesen Sie Berlins beliebtesten Newsletter mit allen wichtigen Nachrichten und Aufregern der Stadt. Gratis. Von Herausgeber Lorenz Maroldt Ich bin damit einverstanden, dass mir per E-Mail, telefonisch oder postalisch interessante Angebote der Tagesspiegel-Gruppe unterbreitet werden. Meine Einwilligung kann ich jederzeit widerrufen. Hier finden Sie unsere Datenschutzerklärung . Jetzt anmelden Berlin verlangte von weiterführenden Schulen noch am Freitag

In [45]:
# Qualitätsprüfung: Text anzeigen artikel_0000.html
text_bvg = df_scraped[df_scraped["filename"].isin(["artikel_0002.html"])]["text"].iloc[0]

# Erste 1.000 Zeichen ausgeben
print(text_bvg[:1000])

© Kitty Kleist-Heinrich Man sei „falsch abgebogen“, sagt Chefin Eva Kreienkamp. Die BVG hatte dagegen geklagt, den Cyber-Sicherheitsstand gegenüber dem Bundesamt nachweisen zu müssen. Stand: 02.04.2021, 16:03 Uhr Der Konflikt schwelt seit Jahren, jetzt ist offenbar ein Ende in Sicht. Die Berliner Verkehrsbetriebe (BVG) geben ihren harten Kurs im Streit mit dem Bundesamt für Sicherheit in der Informationstechnik (BSI) um den Nachweis bestmöglicher Sicherheit gegen Cyberattacken auf. Das Risiko einer Schutzlücke in den IT-Systemen zur Steuerung und Überwachung von U-Bahn, Tram und Bussen scheint nun reduziert zu sein. „Wir haben diesen Donnerstag dem BSI eine Liste mit 23 Mängeln geschickt und damit den Anforderungen Genüge getan“, sagte die BVG-Vorstandsvorsitzende Eva Kreienkamp jetzt dem Tagesspiegel. Die BVG engagierte von sich aus im Februar das Berliner Unternehmen HiSolutions, spezialisiert auf Beratung für „Security und IT-Management“, um die IT-Sicherheit der BVG zu untersuchen.

In [46]:
# Einfacher Qualitätscheck: 2 Artikel je Keyword, Auszug drucken
keywords = ["bvg", "hvv", "mvg", "vhh", "hochbahn"]
max_chars = 800  # Länge des Auszugs

for kw in keywords:
    df_kw = df_scraped[(df_scraped["keyword"].str.lower() == kw) & df_scraped["text"].notna()]
    if df_kw.empty:
        print(f"\n=== {kw.upper()} ===\nKeine Artikel gefunden.")
        continue

    sample = df_kw.sample(n=min(2, len(df_kw)), random_state=42)

    print(f"\n=== {kw.upper()} — {len(df_kw)} Artikel gesamt, Stichprobe {len(sample)} ===")
    for _, row in sample.iterrows():
        print(f"\nDatei: {row.get('filename','?')}")
        print(f"Quelle: {row.get('source','?')}  |  Datum: {row.get('date','?')}")
        print(f"Überschrift: {row.get('headline','?')}")
        print(f"URL: {row.get('url','?')}\n")
        txt = str(row["text"]).replace("\n", " ").strip()
        print(txt[:max_chars])
        print("-" * 80)


=== BVG — 1108 Artikel gesamt, Stichprobe 2 ===

Datei: artikel_1067.html
Quelle: ntv  |  Datum: 2024-01-19
Überschrift: Berlin & BrandenburgNeuer BVG-Chef: Mehr Sauberkeit und stabileren Busverkehr
URL: https://www.n-tv.de/regionales/berlin-und-brandenburg/Neuer-BVG-Chef-Mehr-Sauberkeit-und-stabileren-Busverkehr-article24673550.html

(Foto: Sebastian Christoph Gollnow/dpa) Als früherer Chef der Hamburger Hochbahn hat sich Henrik Falk an saubere Busse und Bahnen gewöhnt. Nun leitet er die Berliner Verkehrsbetriebe und sieht dort nicht nur bei diesem Thema Nachholbedarf. Berlin (dpa/bb) - Von einem verlässlich fahrenden, sauberen öffentlichen Personennahverkehr (ÖPNV) waren die Berliner Verkehrsbetriebe (BVG) zuletzt ein Stück weit entfernt: Aufgrund von zu wenigen Busfahrerinnen und Busfahrern fahren viele Linien seit Monaten nur im reduzierten Takt. Ein hoher Krankenstand führte im Dezember zudem zu Ausfällen und Verspätungen auch bei U- und Straßenbahnen - und so mancher Fahrgast me

In [26]:
# Fehlerhafte Seiten laden
df_scraped_failed = pd.read_csv(FAILED_PATH_RAW_ARTIKEL)

# Überblick
print(df_scraped_failed["error"].value_counts())

error
exception: 'utf-8' codec can't decode byte 0xff in position 21: invalid start byte    3
empty_text                                                                            2
Name: count, dtype: int64


In [27]:
# Textlängen prüfen: Neue Spalte hinzufügen
df_scraped["text_length"] = df_scraped["text"].astype(str).str.len()
print(df_scraped["text_length"].describe())

count     1792.000000
mean      2603.439732
std       3989.846207
min         92.000000
25%        375.000000
50%       1169.000000
75%       3453.500000
max      72345.000000
Name: text_length, dtype: float64


In [28]:
# Auffälligkeiten prüfen: Verteilung lange Texte
df_scraped[df_scraped["text_length"] > 50000][["filename", "text_length"]]

,filename,text_length
750,artikel_0761.html,72345
1267,artikel_1301.html,68010


In [29]:
# Auffälligkeiten prüfen: Text anzeigen artikel_0761.html
text = df_scraped[df_scraped["filename"].isin(["artikel_0761.html"])]["text"].iloc[0]

# Erste 1.000 Zeichen ausgeben
print(text[:1000])

+++ Aktivisten kleben am Sonntag auf Straße am Hauptbahnhof +++ Klimaaktivisten färben Brunnen schwarz +++ Alle Infos im Newsblog Letzte Generation: Erste Klima-Kleberin in Berlin zu Haft ohne Bewährung verurteilt 26.04.2023 Berlin: Letzte Generation bezichtigt andere der Lüge – und was tut sie selbst? 25.04.2023 Berlins Polizeipräsidentin Barbara Slowik hat körperlichen Zwang von Polizisten gegen Klimaaktivisten bei Straßenblockaden verteidigt. Zu Vorwürfen der Polizeigewalt sagte Slowik der Berliner Morgenpost: „Kommt eine Person unseren Aufforderungen, eine Straße zu verlassen, nicht nach, wenden wir gegen sie Maßnahmen des unmittelbaren Zwangs an. Dafür gibt es eine gesetzliche Grundlage, auf der die Polizei, die in diesem Staat das Gewaltmonopol hat, Gewalt anwenden darf.“ Hintergrund sind Videoaufnahmen , in denen ein Polizist einem auf der Straße sitzenden Mann ankündigt, er werde Schmerzen erleiden, falls er die Fahrbahn nicht räume. Anschließend packt der Polizist den Demonstr

#### 5. Bereinigung und Zerlegung der Artikeltexte in Wörter

In [30]:
# Vorbereitung
# Stoppwörter laden
stopwords_list = load_stopwords()

# Funktion: Einen Artikel verarbeiten und Wortfrequenzen zählen
def process_article(row):
    try:
        text = row["text"]
        
        # Sicherstellen, dass ein Text da ist, sonst überspringen
        if pd.isnull(text) or not isinstance(text, str):
            return pd.DataFrame()

        words = clean_and_split_text(text, stopwords_list)

        # Wortfrequenzen zählen
        count = pd.Series(words).value_counts()
        count_df = count.reset_index()
        count_df.columns = ["word", "count"]
        count_df["source"] = row["source"]
        count_df["cluster"] = medium_to_cluster.get(row["source"], "Unbekannt")
        count_df["date"] = row["date"]
        count_df["url"] = row["url"]
        count_df["keyword"] = row.get("keyword", None)
        count_df["year"] = pd.to_datetime(row["date"], errors="coerce").year

        return count_df

    except Exception as e:
        print(f"[WARNING] Fehler bei Artikel {row.get('url', 'unbekannt')}: {e}")
        return pd.DataFrame()

In [31]:
# Verarbeitungsfunktion anwenden
# Artikel laden
df_scraped_final = pd.read_csv(CSV_PATH_RAW_ARTIKEL) 

# Wortfrequenzen sammeln
collection = []
for _, row in df_scraped_final.iterrows():
    count_df = process_article(row)
    if not count_df.empty:
        collection.append(count_df)

# Alle Ergebnisse zusammenführen
if collection:
    df_counts = pd.concat(collection, ignore_index=True)
    df_counts.to_csv(CSV_PATH_COUNTS, index=False)
    print(f"[INFO] Wortfrequenzen mit Keywords gespeichert unter: {CSV_PATH_COUNTS}")
    print("Anzahl verarbeiteter Artikel:", df_scraped_final.shape[0])
else:
    print("[WARNING] Keine Wortfrequenzen berechnet.")

[INFO] Wortfrequenzen mit Keywords gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_counts.csv
Anzahl verarbeiteter Artikel: 1792
